In [799]:
import numpy as np
import warnings
warnings.filterwarnings(action="ignore", category=DeprecationWarning)

In [800]:
MAX_DEF_COUNT_ITER = 10000
"""Максимальное количество итераций по-умолчанию"""
DEF_EPS = 0.0001
"""Точность по-умолчанию"""

# Подготовка

In [801]:
def calc(A: np.matrix, b: np.matrix) -> np.matrix:
    if b is None:
        print("Вектор собственного значения не был посчитан")
        return None
    return float((b.T @ A @ b) / (b.T @ b))

In [802]:
def output(
    A: np.matrix,
    f_1,
    f_2,
    text_1,
    text_2,
    num_iterrations: int = MAX_DEF_COUNT_ITER,
    b_k: np.matrix = None,
    mu: float=None
) -> None:
    abs_v = np.abs(np.linalg.eigvals(A))
    print(
        "Собственные значения матрицы A",
        f"Максимальное: {np.max(abs_v)}",
        f"Минимальное: {np.min(abs_v)}",
        sep="\n",
    )
    print("========")

    print(text_1)
    first_evec = f_1(A=A, num_iterations=num_iterrations, b_k=b_k)
    first_eval = calc(A, first_evec)
    print("========")

    print(text_2)
    if f_2.__name__ != inverse_power_rel.__name__:
        second_evec = f_2(A=A, num_iterations=num_iterrations, b_k=b_k)
        second_eval = calc(A, second_evec)
    else:
        second_evec, second_eval = f_2(A=A, b_k=b_k, mu=mu)
    print("========")

    print(
        "Собственные векторы:",
        f"{text_1}:",
        first_evec,
        f"{text_2}:",
        second_evec,
        sep="\n",
    )
    print(
        "Собственные значения:",
        f"{text_1}: {first_eval}",
        f"{text_2}: {second_eval}",
        sep="\n",
    )

# Прямой метод

$x^{(k)} = \dfrac{Ax^{(k - 1)}}{\alpha_{k - 1}}$

$\lambda_{1}(A) = \dfrac{\left( Ax^{(k)}, x^{(k)} \right)}{(x^{(k)}, x^{(k)})} = \dfrac{(x^{(k)})^{T}A^{(k)}x^{(k)}}{(x^{(k)})^{T}x^{(k)}}$, где $|\lambda_1(A)|$ - наибольшее по модулю СЗ

In [803]:
def power_iteration(A: np.matrix, num_iterations: int = MAX_DEF_COUNT_ITER, b_k: np.matrix = None) -> np.matrix:
    if b_k is None:
        b_k = np.random.rand(A.shape[1], 1)
    print("Начальное приближение:", b_k, sep="\n")

    for _ in range(num_iterations):
        b_k1 = np.dot(A, b_k)

        b_k1_norm = np.linalg.norm(b_k1)

        b_k = b_k1 / b_k1_norm

    return b_k

# Обратный метод

## Стандартная версия

In [804]:
def inverse_power_def(A: np.matrix, num_iterations: int = MAX_DEF_COUNT_ITER, b_k: np.matrix = None) -> np.matrix:
    try:
        inv_A = np.linalg.inv(A)
    except np.linalg.LinAlgError:
        print("Вырожденная матрица")
        return None
    print("Обратная матрица:",inv_A, sep="\n")
    return power_iteration(inv_A, num_iterations, b_k)

In [ ]:
A: np.matrix = np.matrix("1 3 -2 0;"
                         "1 1 4 7;"
                         "4 7 11 23;"
                         "52 66 2 0")  # -0.65
output(A, power_iteration, inverse_power_def, "Прямые итерации", "Обратные итерации (обычные)")

In [ ]:
A: np.matrix = np.matrix("1 -2 -1;"
                         "-1 1 1;"
                         "1 0 -1")  # 0, 2, -1
# Проблема с невырожденной матрицей
output(A, power_iteration, inverse_power_def, "Прямые итерации", "Обратные итерации (обычные)")

## Обратный модифицированный

In [807]:
def inverse_power_mod(
    A: np.matrix, num_iterations: int = MAX_DEF_COUNT_ITER, b_k: np.matrix = None
) -> np.matrix:
    if b_k is None:
        b_k = np.random.rand(A.shape[1], 1)  # Попробовать близкий к настоящему собственный вектор
    print("Начальное приближение:", b_k, sep="\n")

    # print(A, "\n", b_k)
    E = np.eye(A.shape[0], dtype=int)
    mu = 0  # попробовать float
    for iter_index in range(num_iterations):

        # print(A - mu * E)
        inv = np.invert(A - mu * E)

        b_top = inv @ b_k
        # print(b_top)
        c_k = np.linalg.norm(b_top)
        # print(b_top, c_k, b_top / c_k)
        # print(b_top / c_k)

        b_k = b_top / c_k
        # c_k не становится меньше 1
        if c_k < 1:
            break

        # if iter_index % 100 == 0:
        #     print(b_k)
    return b_k

In [ ]:
A: np.matrix = np.matrix("1 -2 -1;"
                         "-1 1 1;"
                         "1 0 -1")  # 0, 2, -1
# Плохо работает без известного начального приближения
output(A, power_iteration, inverse_power_mod, "Прямые итерации", "Обратные итерации (модифицированная)")

## Модификация Релэя

In [809]:
import random


def inverse_power_rel(A, epsilon: float = DEF_EPS, mu = None, b_k = None):
  if b_k is None:
      b_k = np.random.rand(A.shape[1], 1) # Попробовать близкий к настоящему собственный вектор
  if mu is None:
    mu = random.randint(1, 19) / 10 # Случайное начальное приближение
  print("Начальное приближение:", "СЗ:", mu, "СВ:", b_k, sep="\n")
  
  b_k = b_k / np.linalg.norm(b_k)
  y = np.linalg.solve(A - mu * np.eye(A.shape[0]), b_k)
  # print(y)
  # print(x)
  lambda_ = float(np.dot(y.T, b_k))
  # print(lambda_, lambda_ * x)
  mu = mu + 1 / lambda_
  err = np.linalg.norm(y - lambda_ * b_k) / np.linalg.norm(y)

  while err > epsilon:
    b_k = y / np.linalg.norm(y)
    y = np.linalg.solve(A - mu * np.eye(A.shape[0]), b_k)
    lambda_ = float(np.dot(y.T, b_k))
    mu = mu + 1 / lambda_
    err = np.linalg.norm(y - lambda_ * b_k) / np.linalg.norm(y)

  return b_k, float(mu)


In [ ]:
A: np.matrix = np.matrix("1 -2 -1;"
                         "-1 1 1;"
                         "1 0 -1")  # 0, 2, -1
# Сильно зависит от начального приближения
# Добавить анализ через Pandas
# A, calc(A, power_iteration(A, 1000)), inverse_power_rel(A, 0.0001)[1]
output(A, power_iteration, inverse_power_rel, "Прямые итерации", "Обратные итерации (модификация Релэя)")

## Хз, что это

In [811]:
import numpy as np
def inverse_iteration(A, mu = None, max_iterations=10000, tolerance=1e-6):
    if mu is None:
        mu = random.randint(1, 19) / 10
    n = A.shape[0]
    # Начальное приближение для собственного вектора (можно взять случайный вектор)
    v = np.random.rand(n)
    # Нормализация вектора
    v /= np.linalg.norm(v)
    for iteration in range(max_iterations):
        try:
            # Решение линейной системы (A - mu * I)v = w, где I - единичная матрица,
            w = np.linalg.solve(A - mu * np.eye(n), v)
        except np.linalg.LinAlgError:
            print("Сингулярная матрица. Попробуйте использовать другое значение mu.")
            return None, None
        # Нормализация вектора w
        v_new = w / np.linalg.norm(w)
        # Проверка на сходимость
        if np.linalg.norm(v_new - v) < tolerance:
            break
        v = v_new
    # Вычисление приближенного собственного значения,
    eigenvalue = mu + (v.T @ A @ v) / (v.T @ v),
    return v, float(eigenvalue[0][0])

In [ ]:
inverse_iteration(A)